Load required libraries, included conda environment includes all necessary packages + python

In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, hamming_loss
from sklearn.metrics import classification_report

In [2]:
training_data_filepath = "training_data.tsv"

In [3]:
data_cols = [
    'gender',
    'year',
    'course',
    'country',
    'county',
    'commute'
]

input_df = pd.DataFrame(columns = data_cols)

form_df = pd.read_csv(training_data_filepath, sep='\t')
form_df['What sports clubs are you interested in joining?'] = form_df['What sports clubs are you interested in joining?'].fillna('None')
form_df['What Societies are you joining/interested in joining?'] = form_df['What Societies are you joining/interested in joining?'].fillna('None')
form_df['If Ireland or Northern Ireland, what county?'] = form_df['If Ireland or Northern Ireland, what county?'].fillna('International')

input_df['gender'] = form_df['What is your gender?'].values
input_df['year'] = form_df['What year of university are you in?'].values
input_df['course'] = form_df['What course are you doing? (best fit)'].values
input_df['country'] = form_df['What country are you from?'].values
input_df['county'] = form_df['If Ireland or Northern Ireland, what county?'].values
input_df['commute'] = form_df['How far is your commute to UCD?'].values

In [4]:
with open('clubs.txt', 'r') as file:
    clubs_list = file.readlines() 
    clubs_list = [line.strip() for line in clubs_list]

with open('societies.txt', 'r') as file:
    societies_list = file.readlines() 
    societies_list = [line.strip() for line in societies_list]
    
# Remove any newline characters if needed
societies_list = [line.strip() for line in societies_list]

In [5]:
club_df = form_df['What sports clubs are you interested in joining?']

new_df = pd.DataFrame(0, index=form_df.index, columns=clubs_list)

# Iterate over each row in the dataframe
for i, row in form_df.iterrows():
    # Split the string into a list of items
    items = [item.strip() for item in row['What sports clubs are you interested in joining?'].split(',')]
    
    # Set 1 where the item is present in the list
    for item in items:
        if item in clubs_list:
            new_df.at[i, item] = 1
            
input_df = pd.merge(input_df, new_df, left_index=True, right_index=True)

output_data = pd.DataFrame(0, index=form_df.index, columns=societies_list)

# Iterate over each row in the dataframe
for i, row in form_df.iterrows():
    # Split the string into a list of items
    items = [item.strip() for item in row['What Societies are you joining/interested in joining?'].split(',')]
    
    # Set 1 where the item is present in the list
    for item in items:
        if item in societies_list:
            output_data.at[i, item] = 1
        else:
            output_data.at[i, item] = 0

In [6]:
X = input_df
Y = output_data

encoder = OneHotEncoder(drop='first')
X_encoded = encoder.fit_transform(X)
X_encoded_dense = X_encoded.toarray()
X_encoded = encoder.fit_transform(X)
X_encoded_df = pd.DataFrame(X_encoded_dense, columns=encoder.get_feature_names_out(X.columns))
X_encoded = X_encoded_df

X_encoded = pd.merge(X_encoded, new_df, left_index=True, right_index=True)

X_train, X_test, Y_train, Y_test = train_test_split(X_encoded, Y, test_size=0.3, random_state=1)

In [7]:
train_remove_X = [col for col in X_train.columns if (X_train[col] == 0).all()]
test_remove_X = [col for col in X_test.columns if (X_test[col] == 0).all()]
remove_list_X = train_remove_X + test_remove_X
X_train = X_train.drop(columns=remove_list_X)
X_test = X_test.drop(columns=remove_list_X)

train_remove_Y = [col for col in Y_train.columns if (Y_train[col] == 0).all()]
test_remove_Y = [col for col in Y_test.columns if (Y_test[col] == 0).all()]
remove_list_Y = train_remove_Y + test_remove_Y
Y_train = Y_train.drop(columns=remove_list_Y)
Y_test = Y_test.drop(columns=remove_list_Y)

In [8]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=1)
rf_model.fit(X_train, Y_train)
y_pred_proba = rf_model.predict_proba(X_test)

In [9]:
# Create an empty dictionary to store probabilities for each target column
proba_dict = {}

# Dynamically handle any number of target columns
for idx, column in enumerate(Y_train.columns):
    proba_dict[f'{column}'] = y_pred_proba[idx][:, 1]  # Probability for class '1'

# Convert the dictionary to a DataFrame
y_pred_proba_df = pd.DataFrame(proba_dict)
y_pred_proba_df

,Actuarial & Financial Society,Arts Society,Belfield FM Society,Chess Society,Dance Society,Data Society,Dramsoc,Draw Society,Eastern European Society,Elec Engineering Society,...,Nordic Society,Philosophy Society,Politics and International Relations Society,Science Society,Science Fiction & Fantasy Society,Spanish Society,TV Society,Women+ in STEM Society,Volunteers Overseas Student Society,None
0,0.03,0.04,0.04,0.15,0.14,0.74,0.14,0.00,0.03,0.07,...,0.00,0.03,0.10,0.31,0.10,0.04,0.00,0.20,0.07,0.00
1,0.21,0.02,0.00,0.09,0.01,0.77,0.02,0.04,0.00,0.09,...,0.02,0.01,0.00,0.28,0.01,0.00,0.00,0.29,0.03,0.00
2,0.27,0.03,0.03,0.29,0.09,0.80,0.10,0.12,0.00,0.20,...,0.01,0.27,0.06,0.23,0.19,0.02,0.01,0.13,0.32,0.06
3,0.01,0.33,0.08,0.00,0.52,0.95,0.14,0.09,0.07,0.07,...,0.01,0.00,0.05,0.16,0.08,0.00,0.39,0.01,0.48,0.00
4,0.09,0.08,0.00,0.10,0.03,0.92,0.01,0.10,0.00,0.00,...,0.00,0.09,0.01,0.17,0.06,0.00,0.02,0.10,0.12,0.00
5,0.09,0.07,0.09,0.14,0.23,0.92,0.20,0.07,0.11,0.16,...,0.01,0.07,0.10,0.17,0.11,0.03,0.09,0.01,0.26,0.00
6,0.02,0.08,0.01,0.01,0.20,1.00,0.03,0.01,0.00,0.00,...,0.00,0.02,0.02,0.25,0.11,0.00,0.07,0.01,0.20,0.00
7,0.05,0.05,0.04,0.12,0.10,0.89,0.07,0.04,0.01,0.04,...,0.02,0.04,0.01,0.09,0.12,0.04,0.01,0.01,0.19,0.00
8,0.03,0.01,0.00,0.06,0.01,0.71,0.04,0.07,0.00,0.02,...,0.03,0.03,0.01,0.51,0.03,0.00,0.00,0.08,0.42,0.02
9,0.06,0.32,0.25,0.00,0.54,0.92,0.28,0.15,0.01,0.01,...,0.03,0.04,0.00,0.24,0.16,0.22,0.18,0.03,0.59,0.01


In [10]:
# Get binary predictions (0 or 1) directly
y_pred = pd.DataFrame(rf_model.predict(X_test), columns=Y_train.columns, index=Y_test.index)

# Calculate accuracy for each label (column-wise accuracy)
label_accuracies = {col: accuracy_score(Y_test[col], y_pred[col]) for col in Y_train.columns}

# Calculate Hamming loss (fraction of labels that are incorrect)
hamming_loss_value = hamming_loss(Y_test, y_pred)

# Exact match ratio (all labels for a sample must be predicted correctly)
exact_match_ratio = (y_pred == Y_test).mean(axis=1).mean()

# Print results
print("Accuracy for each label:", label_accuracies)
print(f"Hamming Loss: {hamming_loss_value:.4f}")
print(f"Exact Match Ratio: {exact_match_ratio:.4f}")
print(classification_report(Y_test, y_pred, target_names=Y_train.columns))


Accuracy for each label: {'Actuarial & Financial Society': 0.9411764705882353, 'Arts Society': 0.9411764705882353, 'Belfield FM Society': 0.9411764705882353, 'Chess Society': 0.7647058823529411, 'Dance Society': 0.8235294117647058, 'Data Society': 0.8235294117647058, 'Dramsoc': 0.9411764705882353, 'Draw Society': 0.9411764705882353, 'Eastern European Society': 0.8823529411764706, 'Elec Engineering Society': 0.8823529411764706, 'Engineering Society': 0.8235294117647058, 'English & Literary Society': 0.8823529411764706, 'Erasmus Society': 0.7058823529411765, 'Film Society': 0.8235294117647058, 'Food Society': 0.7647058823529411, 'French Society': 0.9411764705882353, 'Games Society': 0.7058823529411765, 'Geography Society': 0.9411764705882353, 'German Society': 0.8823529411764706, 'History Society': 0.9411764705882353, 'Horticulture Society': 0.9411764705882353, 'Indian Society': 0.7058823529411765, 'International Students Society': 0.7058823529411765, 'Internet Society': 0.76470588235294

/opt/miniconda3/envs/DataSoc/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [11]:
user_data = X_encoded.drop(columns=remove_list_X)
user_pred = pd.DataFrame(rf_model.predict(user_data), columns=Y_test.columns)
user_results = user_pred.iloc[-1]
user_results

user_data = X_encoded.drop(columns=remove_list_X)
user_proba = rf_model.predict_proba(user_data)

# Create an empty dictionary to store probabilities for each target column
proba_dict_user = {}

# Dynamically handle any number of target columns
for idx, column in enumerate(Y_train.columns):
    proba_dict_user[f'{column}'] = user_proba[idx][:, 1]  # Probability for class '1'

# Convert the dictionary to a DataFrame
user_proba_df = pd.DataFrame(proba_dict_user)

prob_df = pd.DataFrame(user_proba_df.iloc[-1])
prob_df.columns = ["Predicted Probability"]

report = classification_report(Y_test, y_pred, target_names=Y_train.columns, output_dict=True)
report_df = pd.DataFrame(report).transpose()

label_accuracies_df = pd.DataFrame(list(label_accuracies.items()), columns=['Label', 'Accuracy'])
label_accuracies_df.set_index('Label', inplace=True)

user_output = prob_df.merge(label_accuracies_df, left_index=True, right_index=True).merge(report_df, left_index=True, right_index=True)
user_output["Predicted Probability"] *= 100
user_output = user_output.sort_values(by=["Predicted Probability"], ascending= False)
user_output

/opt/miniconda3/envs/DataSoc/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,Predicted Probability,Accuracy,precision,recall,f1-score,support
Investors & Entrepreneurs Society,60.0,0.882353,0.000000,0.000000,0.000000,2.0
Data Society,34.0,0.823529,0.823529,1.000000,0.903226,14.0
History Society,17.0,0.941176,0.000000,0.000000,0.000000,1.0
German Society,7.0,0.882353,0.000000,0.000000,0.000000,2.0
Games Society,5.0,0.705882,0.500000,0.200000,0.285714,5.0
Internet Society,4.0,0.764706,0.000000,0.000000,0.000000,4.0
Food Society,3.0,0.764706,0.333333,0.333333,0.333333,3.0
Science Society,3.0,0.882353,0.500000,0.500000,0.500000,2.0
Music Society,2.0,0.882353,0.000000,0.000000,0.000000,2.0
Science Fiction & Fantasy Society,2.0,0.882353,0.000000,0.000000,0.000000,2.0
